In [14]:
import ebooklib
from ebooklib import epub
epub_path = '../ebook/J. D. Salinger - The Catcher in the Rye (1951, Penguin Books Ltd).epub'
book = epub.read_epub(epub_path)
chapters = list()
counter = 0
for item in book.get_items():
    if counter < 10:
        counter += 1
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        chapters.append(item.get_content())
chapters[0]

9
9
9
9
9
9
9
9
9
9


b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><div class="calibre1">\n      <p class="calibre2"> THE CATCHER IN THE RYE </p>\n      <p class="calibre3">by J.D. Salinger </p>\n      <p class="calibre3">TO </p>\n      <p class="calibre3">MY </p>\n      <p class="calibre3">MOTHER </p>\n      <div class="mbppagebreak" id="calibre_pb_0"/>\n    </div>\n  </body>\n</html>\n'

In [21]:
from collections import defaultdict

def transform_epub_to_html(epub_path):
    book = epub.read_epub(epub_path)
    chapters = defaultdict(str)
    for item in book.get_items():
        name = item.get_name()
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters[name] = item.get_content()
    return chapters

In [38]:
import re
def transform_html_to_text(chapters):
    blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script']
    for chapter, html in chapters.items():
        chapter_found = re.search(r'The_Catcher_in_the_Rye_split_([\d]{3}).html', chapter)
        if chapter_found:
            chapter = chapter_found.group(1)

            soup = BeautifulSoup(html, 'html.parser')
            text = soup.find_all(text=True)
            for t in text:
                if t.parent.name not in blacklist:
                    output += '{} '.format(t)

In [39]:
transform_html_to_text(chapters)

UnboundLocalError: local variable 'output' referenced before assignment

In [26]:
epub_path = '../ebook/J. D. Salinger - The Catcher in the Rye (1951, Penguin Books Ltd).epub'
chapters = transform_epub_to_html(epub_path)
chapters.keys()

dict_keys(['The_Catcher_in_the_Rye_split_000.html', 'The_Catcher_in_the_Rye_split_001.html', 'The_Catcher_in_the_Rye_split_002.html', 'The_Catcher_in_the_Rye_split_003.html', 'The_Catcher_in_the_Rye_split_004.html', 'The_Catcher_in_the_Rye_split_005.html', 'The_Catcher_in_the_Rye_split_006.html', 'The_Catcher_in_the_Rye_split_007.html', 'The_Catcher_in_the_Rye_split_008.html', 'The_Catcher_in_the_Rye_split_009.html', 'The_Catcher_in_the_Rye_split_010.html', 'The_Catcher_in_the_Rye_split_011.html', 'The_Catcher_in_the_Rye_split_012.html', 'The_Catcher_in_the_Rye_split_013.html', 'The_Catcher_in_the_Rye_split_014.html', 'The_Catcher_in_the_Rye_split_015.html', 'The_Catcher_in_the_Rye_split_016.html', 'The_Catcher_in_the_Rye_split_017.html', 'The_Catcher_in_the_Rye_split_018.html', 'The_Catcher_in_the_Rye_split_019.html', 'The_Catcher_in_the_Rye_split_020.html', 'The_Catcher_in_the_Rye_split_021.html', 'The_Catcher_in_the_Rye_split_022.html', 'The_Catcher_in_the_Rye_split_023.html', 'The_

In [8]:
from bs4 import BeautifulSoup
blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script']
output = ''
soup = BeautifulSoup(chapters[0], 'html.parser')
text = soup.find_all(text=True)
for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

'\n  THE CATCHER IN THE RYE  \n by J.D. Salinger  \n TO  \n MY  \n MOTHER  \n \n \n '